In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
import pandas as pd
from datetime import datetime, timedelta
import time

In [2]:
def split_by_types(info: str) -> tuple:
    start = len("КАТЕГОРИЯ: ")
    last_category = info.find("ИСТЕЦ(ЗАЯВИТЕЛЬ):")
    category = info[start:last_category].strip()
    last_category += len("ИСТЕЦ(ЗАЯВИТЕЛЬ):")
    last_claimant = info.find("ОТВЕТЧИК: ", last_category)
    claimant = info[last_category:last_claimant].strip()
    last_claimant += len("ОТВЕТЧИК: ")
    defendant = info[last_claimant:].strip()
    
    return (category, claimant, defendant)


## Список судов городо МО с населением > 100 000 ч. + Список популярных страховых компаний

In [3]:
link_list = ['https://serpuhov--mo.sudrf.ru/modules.php',
             'https://podolsky--mo.sudrf.ru/modules.php',
             'https://himki--mo.sudrf.ru/modules.php',
             'https://krasnogorsk--mo.sudrf.ru/modules.php',
             'https://balashihinsky--mo.sudrf.ru/modules.php',
             'https://korolev--mo.sudrf.ru/modules.php',
             'https://vidnoe--mo.sudrf.ru/modules.php',
             'https://domodedovo--mo.sudrf.ru/modules.php',
             'https://dolgoprudniy--mo.sudrf.ru/modules.php'
             'https://zhukovsk--mo.sudrf.ru/modules.php',
             'https://kolomna--mo.sudrf.ru/modules.php?name=sud_delo',
             'https://luberetzy--mo.sudrf.ru/modules.php',
             'https://mitishy--mo.sudrf.ru/modules.php',
             'https://odintsovo--mo.sudrf.ru/modules.php',
             'https://orehovo-zuevo--mo.sudrf.ru/modules.php',
             'https://pushkino--mo.sudrf.ru/modules.php',
             'https://ramenskoe--mo.sudrf.ru/modules.php',
             'https://reutov--mo.sudrf.ru/modules.php',
             'https://sergiev-posad--mo.sudrf.ru/modules.php',
             'https://shelkovo--mo.sudrf.ru/modules.php',
             'https://elektrostal--mo.sudrf.ru/modules.php']

Insurance_company = ['АК БАРС Страхование', 'ВТБ Страхование', 'Ингосстрах',
                    'МетЛайф', 'Райффайзен Лайф', 'Сбербанк страхование', 'СОГАЗ',
                    'СОГАЗ-ЖИЗНЬ', 'АИГ', 'АльфаСтрахование', 'РЕСО-Гарантия', 'ВСК',
                    'Ингосстрах-Жизнь', 'ППФ Страхование жизни', 'РСХБ-Страхование',
                    'СиВ Лайф', 'МАКС', 'Ренессанс Страхование', 'Росгосстрах', 'Страхование',
                    'страхование', 'Мегарусс-Д', 'Геополис', 'Страховая бизнес группа', 
                    'Адонис', 'Пари', 'Полис-Гарант', 'Медэкспресс', 'Allianz', 'Либерти',
                    'Зетта', 'ЭРГО', 'ВСК', 'Альянс']

In [6]:
list_kazan = ['http://agryzsky.tat.sudrf.ru/modules.php',
              'http://aznakaevsky.tat.sudrf.ru/modules.php',
              'http://aksubaevsky.tat.sudrf.ru/modules.php',
              'http://aktanyshsky.tat.sudrf.ru/modules.php',
              'http://alekseevsky.tat.sudrf.ru/modules.php',
              'http://alkeevsky.tat.sudrf.ru/modules.php',
              'http://almetevsky.tat.sudrf.ru/modules.php',
              'http://apastovsky.tat.sudrf.ru/modules.php',
              'http://arsky.tat.sudrf.ru/modules.php',
              'http://arsky.tat.sudrf.ru/modules.php',
              'http://atninsky.tat.sudrf.ru/modules.php',
              'http://bavlinsky.tat.sudrf.ru/modules.php',
              'http://baltasinsky.tat.sudrf.ru/modules.php',
              'http://bugulminsky.tat.sudrf.ru/modules.php',
              'http://buinsky.tat.sudrf.ru/modules.php',
              'http://verhne-uslonsky.tat.sudrf.ru/modules.php',
              'http://vysogorsky.tat.sudrf.ru/modules.php',
              'http://drozhanovsky.tat.sudrf.ru/modules.php',
              'http://elabuzhsky.tat.sudrf.ru/modules.php',
              'http://zainsky.tat.sudrf.ru/modules.php',
              'http://zelenodolsky.tat.sudrf.ru/modules.php',
              'http://kaibicky.tat.sudrf.ru/modules.php',
              'http://kamsko-ustinsky.tat.sudrf.ru/modules.php',
              'http://kukmorsky.tat.sudrf.ru/modules.php',
              'http://laishevsky.tat.sudrf.ru/modules.php',
              'http://leninogorsky.tat.sudrf.ru/modules.php',
              'http://mamadyshsky.tat.sudrf.ru/modules.php',
              'http://mendeleevsky.tat.sudrf.ru/modules.php',
              'http://menzelinsky.tat.sudrf.ru/modules.php',
              'http://musliumovsky.tat.sudrf.ru/modules.php',
              'http://naberezhno-chelninsky.tat.sudrf.ru/modules.php',
              'http://nizhnekamsky.tat.sudrf.ru/modules.php',
              'http://novosheshminsky.tat.sudrf.ru/modules.php',
              'http://nurlatsky.tat.sudrf.ru/modules.php',
              'http://pestrechinsky.tat.sudrf.ru/modules.php',
              'http://rybno-slobodsky.tat.sudrf.ru/modules.php',
              'http://sabinsky.tat.sudrf.ru/modules.php',
              'http://sarmanovsky.tat.sudrf.ru/modules.php',
              'http://spassky.tat.sudrf.ru/modules.php',
              'http://tetiushsky.tat.sudrf.ru/modules.php',
              'http://tukaevsky.tat.sudrf.ru/modules.php',
              'http://tiuliachinsky.tat.sudrf.ru/modules.php',
              'http://cheremshansky.tat.sudrf.ru/modules.php',
              'http://chistopolsky.tat.sudrf.ru/modules.php',
              'http://utazinsky.tat.sudrf.ru/modules.php' ]

In [7]:
link_nn = 'http://nizegorodsky.nnov.sudrf.ru/modules.php'
list_nn = ['http://avtozavodsky.nnov.sudrf.ru/modules.php',
           'http://ardatovsky.nnov.sudrf.ru/modules.php',
           'http://arzamassky.nnov.sudrf.ru/modules.php',
           'http://balahninsky.nnov.sudrf.ru/modules.php',
           'http://bogorodsky.nnov.sudrf.ru/modules.php',
           'http://b-boldinsky.nnov.sudrf.ru/modules.php',
           'http://b-murashkinsky.nnov.sudrf.ru/modules.php',
           'http://borsky.nnov.sudrf.ru/modules.php',
           'http://buturlinsky.nnov.sudrf.ru/modules.php',
           'http://vadsky.nnov.sudrf.ru/modules.php',
           'http://varnavinsky.nnov.sudrf.ru/modules.php',
           'http://vachsky.nnov.sudrf.ru/modules.php',
           'http://vetlugsky.nnov.sudrf.ru/modules.php',
           'http://voznesensky.nnov.sudrf.ru/modules.php',
           'http://volodarsky.nnov.sudrf.ru/modules.php',
           'http://vorotinsky.nnov.sudrf.ru/modules.php',
           'http://voskresensky.nnov.sudrf.ru/modules.php',
           'http://viksunsky.nnov.sudrf.ru/modules.php',
           'http://gaginsky.nnov.sudrf.ru/modules.php',
           'http://gorodecsky.nnov.sudrf.ru/modules.php',
           'http://d-konstantinovsky.nnov.sudrf.ru/modules.php',
           'http://dzerginsky.nnov.sudrf.ru/modules.php',
           'http://diveevsky.nnov.sudrf.ru/modules.php',
           'http://kanavinsky.nnov.sudrf.ru/modules.php',
           'http://knyagininsky.nnov.sudrf.ru/modules.php',
           'http://koverninsky.nnov.sudrf.ru/modules.php',
           'http://kr-bakovsky.nnov.sudrf.ru/modules.php',
           'http://kr-oktyabrsky.nnov.sudrf.ru/modules.php',
           'http://kstovsky.nnov.sudrf.ru//modules.php',
           'http://kulebaksky.nnov.sudrf.ru/modules.php',
           'http://leninsky.nnov.sudrf.ru/modules.php',
           'http://lukoyanovsky.nnov.sudrf.ru/modules.php',
           'http://liskovsky.nnov.sudrf.ru/modules.php',
           'http://moskovsky.nnov.sudrf.ru/modules.php',
           'http://navashinsky.nnov.sudrf.ru/modules.php',
           'http://nizegorodsky.nnov.sudrf.ru/modules.php',
           'http://pavlovsky.nnov.sudrf.ru/modules.php',
           'http://pervomaysky.nnov.sudrf.ru/modules.php',
           'http://perevozsky.nnov.sudrf.ru/modules.php',
           'http://pilninsky.nnov.sudrf.ru/modules.php',
           'http://pochinkovsky.nnov.sudrf.ru/modules.php',
           'http://prioksky.nnov.sudrf.ru/modules.php',
           'http://sarovsky.nnov.sudrf.ru/modules.php',
           'http://semenovsky.nnov.sudrf.ru/modules.php',
           'http://sergachsky.nnov.sudrf.ru/modules.php',
           'http://sechenovsky.nnov.sudrf.ru/modules.php',
           'http://sovetsky.nnov.sudrf.ru/modules.php',
           'http://sokolsky.nnov.sudrf.ru/modules.php',
           'http://sormovsky.nnov.sudrf.ru/modules.php',
           'http://sosnovsky.nnov.sudrf.ru/modules.php',
           'http://spassky.nnov.sudrf.ru/modules.php',
           'http://tonkinsky.nnov.sudrf.ru/modules.php',
           'http://tonshaevsky.nnov.sudrf.ru/modules.php',
           'http://urensky.nnov.sudrf.ru/modules.php',
           'http://chkalovsky.nnov.sudrf.ru/modules.php',
           'http://sharangsky.nnov.sudrf.ru/modules.php',
           'http://shatkovsky.nnov.sudrf.ru/modules.php',
           'http://shahunsky.nnov.sudrf.ru/modules.php']

In [4]:
link_R = 'http://voroshilovsky.ros.sudrf.ru/modules.php'
list_R = ['http://voroshilovsky.ros.sudrf.ru/modules.php',
          'http://zheleznodorozhny.ros.sudrf.ru/modules.php',
          'http://kirovsky.ros.sudrf.ru/modules.php',
          'http://leninsky.ros.sudrf.ru/modules.php',
          'http://oktyabrsky.ros.sudrf.ru/modules.php',
          'http://pervomajsky.ros.sudrf.ru/modules.php',
          'http://proletarsky.ros.sudrf.ru/modules.php',
          'http://sovetsky.ros.sudrf.ru/modules.php',
          'http://azovsky.ros.sudrf.ru/modules.php',
          'http://aksajsky.ros.sudrf.ru/modules.php',
          'http://bagaevsky.ros.sudrf.ru/modules.php',
          'http://batajsky.ros.sudrf.ru/modules.php',
          'http://belokalitvensky.ros.sudrf.ru/modules.php',
          'http://volgodonskoy.ros.sudrf.ru/modules.php',
          'http://gukovsky.ros.sudrf.ru/modules.php',
          'http://donecky.ros.sudrf.ru/modules.php',
          'http://egorlyksky.ros.sudrf.ru/modules.php',
          'http://zernogradsky.ros.sudrf.ru/modules.php',
          'http://zimovnikovsky.ros.sudrf.ru/modules.php',
          'http://kamensky.ros.sudrf.ru/modules.php',
          'http://krasnosulinsky.ros.sudrf.ru/modules.php',
          'http://martinovsky.ros.sudrf.ru/modules.php',
          'http://mkurgansky.ros.sudrf.ru/modules.php',
          'http://millerovsky.ros.sudrf.ru/modules.php',
          'http://morozovsky.ros.sudrf.ru/modules.php',
          'http://myasnikovsky.ros.sudrf.ru/modules.php',
          'http://neklinovsky.ros.sudrf.ru/modules.php',
          'http://novocherkassky.ros.sudrf.ru/modules.php',
          'http://novoshahtinsky.ros.sudrf.ru/modules.php',
          'http://oblivsky.ros.sudrf.ru/modules.php',
          'http://oktyabrskys.ros.sudrf.ru/modules.php',
          'http://orlovsky.ros.sudrf.ru/modules.php',
          'http://peschanokopsky.ros.sudrf.ru/modules.php',
          'http://proletarskys.ros.sudrf.ru/modules.php',
          'http://remontnensky.ros.sudrf.ru/modules.php',
          'http://salsky.ros.sudrf.ru/modules.php',
          'http://semikarakorsky.ros.sudrf.ru/modules.php',
          'http://taganrogsky.ros.sudrf.ru/modules.php',
          'http://ustdonetsky.ros.sudrf.ru/modules.php',
          'http://celinsky.ros.sudrf.ru/modules.php',
          'http://cimlyansky.ros.sudrf.ru/modules.php',
          'http://chertkovsky.ros.sudrf.ru/modules.php',
          'http://shahtinsky.ros.sudrf.ru/modules.php',
          'http://sholohonsky.ros.sudrf.ru/modules.php']

In [9]:
link_Ekb = 'http://leninskyeka.svd.sudrf.ru/modules.php'
list_Ekb = ['http://verhisetsky.svd.sudrf.ru/modules.php',
            'http://zheleznodorozhny.svd.sudrf.ru/modules.php',
            'http://kirovsky.svd.sudrf.ru/modules.php',
            'http://leninskyeka.svd.sudrf.ru/modules.php',
            'http://oktiabrsky.svd.sudrf.ru/modules.php',
            'http://ordzhonikidzevsky.svd.sudrf.ru/modules.php',
            'http://chkalovsky.svd.sudrf.ru/modules.php',
            'http://artinsky.svd.sudrf.ru/modules.php',
            'http://baikalovsky.svd.sudrf.ru/modules.php',
            'http://bogdanovichsky.svd.sudrf.ru/modules.php',
            'http://verhotursky.svd.sudrf.ru/modules.php',
            'http://ivdelsky.svd.sudrf.ru/modules.php',
            'http://kamyshlovsky.svd.sudrf.ru/modules.php',
            'http://karpinsky.svd.sudrf.ru/modules.php',
            'http://kachkanarsky.svd.sudrf.ru/modules.php',
            'http://kirovgradsky.cvd.sudrf.ru/modules.php',
            'http://krasnouralsky.svd.sudrf.ru/modules.php',
            'http://kushvinsky.svd.sudrf.ru/modules.php',
            'http://nizhneserginsky.svd.sudrf.ru/modules.php',
            'http://nizhneturinsky.svd.sudrf.ru/modules.php',
            'http://novolialinsky.svd.sudrf.ru/modules.php',
            'http://rezhevskoy.svd.sudrf.ru/modules.php',
            'http://severouralsky.svd.sudrf.ru/modules.php',
            'http://slobodo-turinsky.svd.sudrf.ru/modules.php',
            'http://suholozhsky.svd.sudrf.ru/modules.php',
            'http://tavdinsky.svd.sudrf.ru/modules.php',
            'http://talicky.svd.sudrf.ru/modules.php',
            'http://tugulymsky.svd.sudrf.ru/modules.php',
            'http://turinsky.svd.sudrf.ru/modules.php',
            'http://shalinsky.svd.sudrf.ru/modules.php']

In [10]:
link_Vla = 'http://pervorechensky.prm.sudrf.ru/modules.php'

list_Vla = ['http://anuchinsky.prm.sudrf.ru/modules.php',
            'http://arsenievsky.prm.sudrf.ru/modules.php',
            'http://artemovsky.prm.sudrf.ru/modules.php',
            'http://dalnegorsky.prm.sudrf.ru/modules.php',
            'http://dalnerechensky.prm.sudrf.ru/modules.php',
            'http://kavalerovsky.prm.sudrf.ru/modules.php',
            'http://kirovsky.prm.sudrf.ru/modules.php',
            'http://krasnoarmeysky.prm.sudrf.ru/modules.php',
            'http://lazovskiy.prm.sudrf.ru/modules.php',
            'http://leninsky.prm.sudrf.ru/modules.php',
            'http://lesozavodsky.prm.sudrf.ru/modules.php',
            'http://mihaylovsky.prm.sudrf.ru/modules.php',
            'http://nadezhdinsky.prm.sudrf.ru/modules.php',
            'http://nahodkinsky.prm.sudrf.ru/modules.php',
            'http://oktiabrsky.prm.sudrf.ru/modules.php',
            'http://partizansky.prm.sudrf.ru/modules.php',
            'http://pervomaysky.prm.sudrf.ru/modules.php',
            'http://pervorechensky.prm.sudrf.ru/modules.php',
            'http://pogranichny.prm.sudrf.ru/modules.php',
            'http://pozharsky.prm.sudrf.ru/modules.php',
            'http://sovetsky.prm.sudrf.ru/modules.php',
            'http://spassky.prm.sudrf.ru/modules.php',
            'http://terneysky.prm.sudrf.ru/modules.php',
            'http://ussuriysky.prm.sudrf.ru/modules.php',
            'http://fokinsky.prm.sudrf.ru/modules.php',
            'http://frunzensky.prm.sudrf.ru/modules.php',
            'http://hankaisky.prm.sudrf.ru/modules.php',
            'http://hasansky.prm.sudrf.ru/modules.php',
            'http://horolsky.prm.sudrf.ru/modules.php',
            'http://chernigovsky.prm.sudrf.ru/modules.php',
            'http://chuguevsky.prm.sudrf.ru/modules.php',
            'http://shkotovsky.prm.sudrf.ru/modules.php',
            'http://yakovlevsky.prm.sudrf.ru/modules.php'] 

In [11]:
link_spb = 'http://sankt-peterburgsky.spb.sudrf.ru/modules.php'
list_spb = ['http://sankt-peterburgsky.spb.sudrf.ru/modules.php',
            'http://vos.spb.sudrf.ru/modules.php',
            'http://vbr.spb.sudrf.ru/modules.php',
            'http://dzr.spb.sudrf.ru/modules.php',
            'http://zgr.spb.sudrf.ru/modules.php',
            'http://kln.spb.sudrf.ru/modules.php',
            'http://krv.spb.sudrf.ru/modules.php',
            'http://klp.spb.sudrf.ru/modules.php',
            'http://kgv.spb.sudrf.ru/modules.php',
            'http://ksl.spb.sudrf.ru/modules.php',
            'http://krn.spb.sudrf.ru/modules.php',
            'http://kbs.spb.sudrf.ru/modules.php',
            'http://lnn.spb.sudrf.ru/modules.php',
            'http://msk.spb.sudrf.ru/modules.php',
            'http://nvs.spb.sudrf.ru/modules.php',
            'http://oktibrsky.spb.sudrf.ru/modules.php',
            'http://pgr.spb.sudrf.ru/modules.php',
            'http://pdv.spb.sudrf.ru/modules.php',
            'http://primorsky.spb.sudrf.ru/modules.php',
            'http://psh.spb.sudrf.ru/modules.php',
            'http://srt.spb.sudrf.ru/modules.php',
            'http://smolninsky.spb.sudrf.ru/modules.php',
            'http://frn.spb.sudrf.ru/modules.php',
            'http://oblsud.lo.sudrf.ru/modules.php',
            'http://boksiti.lo.sudrf.ru/modules.php',
            'http://volosov.lo.sudrf.ru/modules.php',
            'http://volhovsky.lo.sudrf.ru/modules.php',
            'http://vsevgorsud.lo.sudrf.ru/modules.php',
            'http://vyborgsky.lo.sudrf.ru/modules.php',
            'http://gatchinsky.lo.sudrf.ru/modules.php',
            'http://kingisepp.lo.sudrf.ru/modules.php',
            'http://kirishsky.lo.sudrf.ru/modules.php',
            'http://kirovsky.lo.sudrf.ru/modules.php',
            'http://lodeynopolsky.lo.sudrf.ru/modules.php',
            'http://lomonosovsky.lo.sudrf.ru/modules.php',
            'http://lugasud.lo.sudrf.ru/modules.php',
            'http://podporozhsky.lo.sudrf.ru/modules.php',
            'http://priozersky.lo.sudrf.ru/modules.php',
            'http://slancevsky.lo.sudrf.ru/modules.php',
            'http://sosnovoborsky.lo.sudrf.ru/modules.php',
            'http://tikhvin.lo.sudrf.ru/modules.php',
            'http://tosnensky.lo.sudrf.ru/modules.php']

## Ввод начальной и конечной даты интервала рассматриваемых судебных дел 

1. Даты вводить в указанном формате
2. Интервал порядка недели обрабатывается примерно 1 минуту

In [27]:
date_first = input('Enter a first date in DD.MM.YYYY format:  ')
date_last = input('Enter a last date in DD.MM.YYYY format:  ')
date_format = "%d.%m.%Y"
date_1 = datetime.strptime(date_first, date_format)
date_n = datetime.strptime(date_last, date_format)
day_delta = timedelta(days=1)

Enter a first date in DD.MM.YYYY format:  21.11.2018
Enter a last date in DD.MM.YYYY format:  31.12.2018


### Парсинг информации с учётом фильтра:  Категория == Дела, связанные с ОСАГО AND Ответчик == Страховая Компания (т.е. 1-ое + 2-ое условие вместе)

In [28]:
def OSAGO_R(date_1, date_n, day_delta): 
    start = time.monotonic()
    data_list = []
    for i in range((date_n - date_1).days):
        print((date_1 + i*day_delta).strftime("%d.%m.%Y"))
        if i%2 == 0:
            time.sleep(0.15)
        for link in list_R:
            page = requests.get(link, params={'name':'sud_delo', 'srv_num': '1', 'H_date': (date_1 + i*day_delta).strftime("%d.%m.%Y")}, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}, verify=False)
            soup = BeautifulSoup(page.content)
            try:
                table = soup.find('div', id='tablcont')
                trs = table.find_all('tr')[2:]
                for tr in trs:
                    tds = tr.find_all('td')
                    if len(tds) > 1 and not (tds[4].getText().find('КАТЕГОРИЯ' and 'ИСТЕЦ' and 'ОТВЕТЧИК') == -1):
                        info = tds[4].getText()
                        info_1, info_2, info_3 = split_by_types(info)
                        flag = False
                        for elem in Insurance_company:
                            if not (info_3.find(elem) == -1):
                                flag = True
                        if not (info_1.find('ОСАГО') == -1) and flag:        
                            place = soup.find('title').getText()
                            number = tds[1].getText()   
                            judge = tds[5].getText()
                            res = tds[6].getText()
                            data_list.append({
                                'place' : place, 
                                'date' : (date_1 + i*day_delta).strftime("%d.%m.%Y"),
                                'id': number,
                                'category' : info_1,
                                'clainmant' : info_2,
                                'defendant' : info_3,
                                'name_judge' : judge,
                                'result' : res
                            })
                            df_R = pd.DataFrame(data_list)
                            df_R.to_excel('Rostov_2018_8.xlsx')
            except AttributeError:
                pass
    #df_Spb = pd.DataFrame(data_list)
    #df_Spb.to_excel('Spb_info_3.xlsx')
    finish = time.monotonic()
    result = finish - start
    print("Время выполнения: ", str(result), "сек.")
    return df_Spb

In [29]:
import warnings

warnings.filterwarnings('ignore')
df = OSAGO_R(date_1, date_n, day_delta)

21.11.2018
22.11.2018
23.11.2018
24.11.2018
25.11.2018
26.11.2018
27.11.2018
28.11.2018
29.11.2018
30.11.2018
01.12.2018
02.12.2018
03.12.2018
04.12.2018
05.12.2018
06.12.2018
07.12.2018
08.12.2018
09.12.2018
10.12.2018
11.12.2018
12.12.2018
13.12.2018
14.12.2018
15.12.2018
16.12.2018
17.12.2018
18.12.2018
19.12.2018
20.12.2018
21.12.2018
22.12.2018
23.12.2018
24.12.2018
25.12.2018
26.12.2018
27.12.2018
28.12.2018
29.12.2018
30.12.2018
Время выполнения:  2925.282318124002 сек.


NameError: name 'df_Spb' is not defined

In [41]:
df

,place,date,id,category,clainmant,defendant,name_judge,result
0,Санкт-Петербургский городской суд,01.04.2021,33-7437/2021,"Иски, связанные с возмещением ущерба → О взыск...",Кутяшев Артём Викторович,АО Тинькофф СтрахованиеСуд (мировой судья) пер...,Бучнева Олеся Ивановна,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
1,Василеостровский районный суд города Санкт-Пет...,01.04.2021,2-1409/2021 (2-5007/2020;) ~ М-4465/2020,"Иски, связанные с возмещением ущерба → О взыск...",Ежов Кирилл Владимирович,"Акционерное общество ""Группа Ренессанс Страхов...",Дугина Наталья Владимировна,"Иск (заявление, жалоба) оставлены без рассмотр..."
2,Ленинский районный суд города Санкт-Петербурга,01.04.2021,2-570/2021 (2-2171/2020;) ~ М-2173/2020,"Иски, связанные с возмещением ущерба → О взыск...",Теули Екатерина Ивановна,ОАО Альфа-Страхование,Батогов Александр Владимирович,Производство по делу прекращено
3,Московский районный суд города Санкт-Петербурга,01.04.2021,2-1233/2021 (2-7576/2020;) ~ М-6222/2020,"Иски, связанные с возмещением ущерба → О взыск...",Мацарский Евгений Александрович,"САО ""РЕСО-Гарантия""",Тиунова Ольга Николаевна,Рассмотрение дела начато с начала
4,Петроградский районный суд города Санкт-Петерб...,01.04.2021,2-1516/2021 ~ М-243/2021,"Иски, связанные с возмещением ущерба → О взыск...",Дмитриева Екатерина Евгеньевна,"СПАО ""Ингосстрах""",Никитин Сергей Сергеевич,Объявлен перерыв
5,Петроградский районный суд города Санкт-Петерб...,01.04.2021,2-1497/2021 ~ М-245/2021,"Иски, связанные с возмещением ущерба → О взыск...",Алексеев Алексей Владимирович,"СПАО ""Ингосстрах""",Никитин Сергей Сергеевич,"Иск (заявление, жалоба) оставлены без рассмотр..."
6,Приморский районный суд города Санкт-Петербурга,01.04.2021,2-3206/2021 (2-10727/2020;) ~ М-8159/2020,"Иски, связанные с возмещением ущерба → О взыск...",Яковенко Виктор Викторович,"ПАО ""СК Росгосстрах"", Попов Максим Дмитриевич",Карпенкова Наталия Евгеньевна,Заседание отложено
7,Приморский районный суд города Санкт-Петербурга,01.04.2021,2-3713/2021 (2-11282/2020;) ~ М-8331/2020,"Иски, связанные с возмещением ущерба → О взыск...",Верещагин Евгений Александрович,"ПАО СК ""Росгосстрах""",Карпенкова Наталия Евгеньевна,Заседание отложено
8,Пушкинский районный суд города Санкт-Петербурга,01.04.2021,2-1065/2021 (2-4425/2020;) ~ М-3304/2020,"Иски, связанные с возмещением ущерба → О взыск...","САО ""ВСК""","Прокофичев Николай Николаевич, СПАО «РЕСО-Гара...",Яхонтова Надежда Сергеевна,Вынесено решение по делу


In [ ]:
df.to_excel('Spb_info.xlsx')
from google.colab import files
files.download("Spb_info.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
df_1 = pd.read_excel('Spb_info_1.xlsx')
df_1

,category,clainmant,date,defendant,id,name_judge,place,result
0,"Иски, связанные с возмещением ущерба → О взыск...",Тамразова Джульетта Аршаковна,09.01.2019,"ОАО ""АльфаСтрахование""",2-50/2019 (2-2363/2018;) ~ М-1279/2018,Прозорова Елена Вадимовна,Василеостровский районный суд города Санкт-Пет...,Производство по делу приостановлено
1,"Иски, связанные с возмещением ущерба → О взыск...",Лебедева Ирина Павловна,09.01.2019,"АО ""Группа Ренессанс страхование""",2-1634/2019 (2-7780/2018;) ~ М-6289/2018,Метелкина Антонина Юрьевна,Московский районный суд города Санкт-Петербурга,Назначено судебное заседание
2,"Иски, связанные с возмещением ущерба → О взыск...",Алескерова Роза Исмаиловна,09.01.2019,"ПАО СК ""Росгосстрах""",11-2/2019 (11-115/2018;),Пазюченко Ирина Жоржовна,Октябрьский районный суд города Санкт-Петербурга,Дело окончено
3,"Иски, связанные с возмещением ущерба → О взыск...",Горохова Солмаз Эйвазовна,09.01.2019,"ПАО СК «Росгосстрах», Шварц Алексей Александрович",2-16/2019 (2-3432/2018; 2-14052/2017;) ~ М-118...,Тумасян Каринэ Левоновна,Приморский районный суд города Санкт-Петербурга,Заседание отложено
4,"Иски, связанные с возмещением ущерба → О взыск...",Мурская Ксения Вадимовна,09.01.2019,"ПАО СК ""Росгосстрах""",2-21/2019 (2-3710/2018; 2-14348/2017;) ~ М-115...,Мозерова Татьяна Михайловна,Приморский районный суд города Санкт-Петербурга,Вынесено решение по делу
5,"Иски, связанные с возмещением ущерба → О взыск...",Богданов Ю.С.,10.01.2019,"ООО ""Группа Ренессанс Страхование""Суд (мировой...",33-486/2019 (33-25834/2018;),Савельева Татьяна Юрьевна,Санкт-Петербургский городской суд,другие апелляционные определения с удовлетворе...
6,"Иски, связанные с возмещением ущерба → О взыск...",Велиева Э.М.,10.01.2019,"СПАО ""РЕСО-Гарантия""Суд (мировой судья) первой...",33-732/2019 (33-26404/2018;),Савельева Татьяна Юрьевна,Санкт-Петербургский городской суд,ОПРЕДЕЛЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
7,"Иски, связанные с возмещением ущерба → О взыск...",Жустров Юрий Аркадьевич,10.01.2019,"ПАО СК ""Росгосстрах""Суд (мировой судья) первой...",33-479/2019 (33-25818/2018;),Птохова Злата Юрьевна,Санкт-Петербургский городской суд,решение (не осн. требов.) изменено (без направ...
8,"Иски, связанные с возмещением ущерба → О взыск...",Котова М.А.,10.01.2019,"АО ""Либерти страхование""Суд (мировой судья) пе...",33-333/2019 (33-24990/2018;),Птохова Злата Юрьевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
9,"Иски, связанные с возмещением ущерба → О взыск...",Степанян В.А.,10.01.2019,"ПАО СМК ""Росгосстрах""Суд (мировой судья) перво...",33-959/2019 (33-26776/2018;),Александрова Юлия Кирилловна,Санкт-Петербургский городской суд,решение (осн. требов.) отменено полностью с вы...


In [27]:
df_2 = pd.read_excel('Spb_info_2.xlsx')
df_2

,place,date,id,category,clainmant,defendant,name_judge,result
0,Санкт-Петербургский городской суд,01.04.2019,33-8800/2019,"Иски, связанные с возмещением ущерба → О взыск...",Смирнов Александр Михайлович,ООО Группа Ренессанс СтрахованиеСуд (мировой с...,Медведкина Виктория Александровна,ОПРЕДЕЛЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
1,Санкт-Петербургский городской суд,01.04.2019,33-7747/2019,"Иски, связанные с возмещением ущерба → О взыск...",Лонский Андрей Александрович,АО СОГАЗСуд (мировой судья) первой инстанции: ...,Медведкина Виктория Александровна,решение (осн. требов.) отменено в части с выне...
2,Василеостровский районный суд города Санкт-Пет...,01.04.2019,2-1405/2019 ~ М-215/2019,"Иски, связанные с возмещением ущерба → О взыск...",Смирнов Александр Александрович,"АО ""МАКС""",Дугина Наталья Владимировна,Назначено судебное заседание
3,Василеостровский районный суд города Санкт-Пет...,01.04.2019,2-507/2019 (2-4349/2018;) ~ М-3573/2018,"Иски, связанные с возмещением ущерба → О взыск...",Гусева Нина Павловна,"ООО ""Группа Ренессанс Страхование""",Дугина Наталья Владимировна,Вынесено решение по делу
4,Калининский районный суд города Санкт-Петербурга,01.04.2019,2-3744/2019 ~ М-1166/2019,"Иски, связанные с возмещением ущерба → О взыск...",Бицман Тамара Григорьевна,СПАО «РЕСО-Гарантия»,Смирнова О.А.,Назначено судебное заседание
5,Красногвардейский районный суд города Санкт-Пе...,01.04.2019,2-2015/2019 ~ М-396/2019,"Иски, связанные с возмещением ущерба → О взыск...",Кицын Кирилл Михайлович,"АО ""МАКС""",Рябинин Алексей Николаевич,Назначено судебное заседание
6,Красносельский районный суд города Санкт-Петер...,01.04.2019,2-2374/2019 ~ М-930/2019,"Иски, связанные с возмещением ущерба → О взыск...",Дошина Капитолина Сергеевна,"СПАО ""РЕСО-Гарантия""",Полинова Т.А.,Производство по делу приостановлено
7,Ленинский районный суд города Санкт-Петербурга,01.04.2019,2-709/2019 ~ М-444/2019,"Иски, связанные с возмещением ущерба → О взыск...",Додов Дауд Бамат-Гиреевич,"АО ""АльфаСтрахование""",Ткачева Ольга Сергеевна,Производство по делу приостановлено
8,Московский районный суд города Санкт-Петербурга,01.04.2019,2-496/2019 (2-5835/2018;) ~ М-3471/2018,"Иски, связанные с возмещением ущерба → О взыск...",Мичурин Вячеслав Дмитриевич,"АО ""Группа Ренессанс Страхование""",Тиунова Ольга Николаевна,Объявлен перерыв
9,Петроградский районный суд города Санкт-Петерб...,01.04.2019,2-422/2019 (2-4426/2018;) ~ М-3913/2018,"Иски, связанные с возмещением ущерба → О взыск...",Овчинников Роман Николаевич,"СПАО ""Ингосстрах""",Галкина Елена Сергеевна,Производство по делу приостановлено


In [28]:
df_7 = pd.read_excel('Spb_info_7.xlsx')
df_7

,category,clainmant,date,defendant,id,name_judge,place,result
0,"Иски, связанные с возмещением ущерба → О взыск...",Купцова Вилена Иосифовна,01.07.2019,АО Альфа СтрахованиеСуд (мировой судья) перво...,33-11659/2019,Мелешко Наталья Владимировна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
1,"Иски, связанные с возмещением ущерба → О взыск...",Жуков Сергей Борисович,01.07.2019,"СПАО ""РЕСО-Гарантия""",2-767/2019 (2-5917/2018;) ~ М-5598/2018,Рябинин Алексей Николаевич,Красногвардейский районный суд города Санкт-Пе...,Вынесено решение по делу
2,"Иски, связанные с возмещением ущерба → О взыск...",Ямаков Рустам Шамильевич,01.07.2019,"ООО ""Группа Ренессанс Страхование"", ПАО СК ""Ро...",2-2412/2018 ~ М-704/2018,Овчаров Виктор Викторович,Красносельский районный суд города Санкт-Петер...,Ходатайство/заявление УДОВЛЕТВОРЕНО
3,"Иски, связанные с возмещением ущерба → О взыск...",Кчибекова Изольда Лермонтовна,01.07.2019,АО «Либерти Страхование»,2-28/2020 (2-2642/2019;) ~ М-1238/2019,Христосова Анна Игоревна,Красносельский районный суд города Санкт-Петер...,Производство по делу приостановлено
4,"Иски, связанные с возмещением ущерба → О взыск...",Кузнецов Станислав Юрьевич,01.07.2019,"АО ""Группа Ренессанс Страхование"", Скородумов ...",2-3390/2019 ~ М-1055/2019,Смирнова Елена Валерьевна,Московский районный суд города Санкт-Петербурга,Вынесено решение по делу
5,"Иски, связанные с возмещением ущерба → О взыск...",Териев Хизир Исхажиевич,01.07.2019,"АО ""Группа Ренессанс Страхование"", Конышева Ел...",2-5716/2019 ~ М-2274/2019,Бурданова Наталия Анатольевна,Московский районный суд города Санкт-Петербурга,Заседание отложено
6,"Иски, связанные с возмещением ущерба → О взыск...",Воробьев Евгений Александрович,01.07.2019,"АО ""МАКС""",2-1912/2019 (2-8230/2018;) ~ М-7963/2018,Игнатьева Анна Андреевна,Невский районный суд города Санкт-Петербурга,Заседание отложено
7,"Иски, связанные с возмещением ущерба → О взыск...",Швецова Андрей Константинович,01.07.2019,"АО ""Тинькофф Страхование""",2-5286/2019 ~ М-2700/2019,Игнатьева Анна Андреевна,Невский районный суд города Санкт-Петербурга,Рассмотрение дела начато с начала
8,"Иски, связанные с возмещением ущерба → О взыск...",Оленич Алла Михайловна,01.07.2019,"ПАО СК ""Росгосстрах""",2-3070/2019 ~ М-430/2019,Попова Наталья Владимировна,Невский районный суд города Санкт-Петербурга,Вынесено решение по делу
9,"Иски, связанные с возмещением ущерба → О взыск...",Владимиров Александр Иванович,01.07.2019,"АО ""НАСКО"" в лице филиала в Санкт-Петербурге.,...",2-3035/2019 ~ М-908/2019,Ерунова Елена Владиславовна,Невский районный суд города Санкт-Петербурга,Рассмотрение дела начато с начала


In [29]:
df_8 = pd.read_excel('Spb_info_8.xlsx')
df_8

,category,clainmant,date,defendant,id,name_judge,place,result
0,"Иски, связанные с возмещением ущерба → О взыск...",Мухач Алексей Владимирович,01.08.2019,ПАО СК «Росгосстрах»Суд (мировой судья) первой...,33-16806/2019,Литвинова Ирина Анатольевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
1,"Иски, связанные с возмещением ущерба → О взыск...",Симачков Сергей Сергеевич,01.08.2019,СПАО Ингосстрах в лице филиала в СПБСуд (миров...,33-18221/2019,Литвинова Ирина Анатольевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
2,"Иски, связанные с возмещением ущерба → О взыск...",Еремян Геворг Арменович,01.08.2019,ПАО СК РосгосстрахСуд (мировой судья) первой и...,33-18424/2019,Литвинова Ирина Анатольевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
3,"Иски, связанные с возмещением ущерба → О взыск...",Фомина Наталья Александровна,01.08.2019,САО ВСКСуд (мировой судья) первой инстанции: К...,33-18483/2019,Литвинова Ирина Анатольевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
4,"Иски, связанные с возмещением ущерба → О взыск...",Кривогин Павел Викторович,01.08.2019,СПАО РЕСО-ГарантияСуд (мировой судья) первой и...,33-18423/2019,Мирошникова Елена Николаевна,Санкт-Петербургский городской суд,определение отменено полностью с разрешением в...
5,"Иски, связанные с возмещением ущерба → О взыск...",Аникин Илья Николаевич,01.08.2019,ПАО СК РосгосстрахСуд (мировой судья) первой и...,33-17445/2019,Мирошникова Елена Николаевна,Санкт-Петербургский городской суд,решение (не осн. требов.) изменено (без направ...
6,"Иски, связанные с возмещением ущерба → О взыск...",Ага-Мурат Тимур Исламович,01.08.2019,ПАО СК «Росгосстрах»Суд (мировой судья) первой...,33-17446/2019,Мирошникова Елена Николаевна,Санкт-Петербургский городской суд,решение (не осн. требов.) изменено (без направ...
7,"Иски, связанные с возмещением ущерба → О взыск...",Андронова Полина Юрьевна,01.08.2019,"СПАО ""Ингосстрах""",2-3897/2019 ~ М-2692/2019,Лебедева Александра Станиславовна,Кировский районный суд города Санкт-Петербурга,"Иск (заявление, жалоба) оставлены без рассмотр..."
8,"Иски, связанные с возмещением ущерба → О взыск...",Александрович Валентин Петрович,01.08.2019,"ООО ""СК АНГАРА"", ООО «Группа Ренессанс Страхов...",2-2246/2019 ~ М-833/2019,Лебедева Александра Станиславовна,Кировский районный суд города Санкт-Петербурга,Рассмотрение дела начато с начала
9,"Иски, связанные с возмещением ущерба → О взыск...",Потапова Алиса Андреевна,01.08.2019,"СПАО ""РЕСО-Гарантия""",2-535/2018 (2-6702/2017;) ~ М-6879/2017,Семенова Ольга Алексеевна,Невский районный суд города Санкт-Петербурга,Отложено


In [30]:
df_8 = pd.read_excel('Spb_info_8.xlsx')
df_8

,category,clainmant,date,defendant,id,name_judge,place,result
0,"Иски, связанные с возмещением ущерба → О взыск...",Мухач Алексей Владимирович,01.08.2019,ПАО СК «Росгосстрах»Суд (мировой судья) первой...,33-16806/2019,Литвинова Ирина Анатольевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
1,"Иски, связанные с возмещением ущерба → О взыск...",Симачков Сергей Сергеевич,01.08.2019,СПАО Ингосстрах в лице филиала в СПБСуд (миров...,33-18221/2019,Литвинова Ирина Анатольевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
2,"Иски, связанные с возмещением ущерба → О взыск...",Еремян Геворг Арменович,01.08.2019,ПАО СК РосгосстрахСуд (мировой судья) первой и...,33-18424/2019,Литвинова Ирина Анатольевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
3,"Иски, связанные с возмещением ущерба → О взыск...",Фомина Наталья Александровна,01.08.2019,САО ВСКСуд (мировой судья) первой инстанции: К...,33-18483/2019,Литвинова Ирина Анатольевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
4,"Иски, связанные с возмещением ущерба → О взыск...",Кривогин Павел Викторович,01.08.2019,СПАО РЕСО-ГарантияСуд (мировой судья) первой и...,33-18423/2019,Мирошникова Елена Николаевна,Санкт-Петербургский городской суд,определение отменено полностью с разрешением в...
5,"Иски, связанные с возмещением ущерба → О взыск...",Аникин Илья Николаевич,01.08.2019,ПАО СК РосгосстрахСуд (мировой судья) первой и...,33-17445/2019,Мирошникова Елена Николаевна,Санкт-Петербургский городской суд,решение (не осн. требов.) изменено (без направ...
6,"Иски, связанные с возмещением ущерба → О взыск...",Ага-Мурат Тимур Исламович,01.08.2019,ПАО СК «Росгосстрах»Суд (мировой судья) первой...,33-17446/2019,Мирошникова Елена Николаевна,Санкт-Петербургский городской суд,решение (не осн. требов.) изменено (без направ...
7,"Иски, связанные с возмещением ущерба → О взыск...",Андронова Полина Юрьевна,01.08.2019,"СПАО ""Ингосстрах""",2-3897/2019 ~ М-2692/2019,Лебедева Александра Станиславовна,Кировский районный суд города Санкт-Петербурга,"Иск (заявление, жалоба) оставлены без рассмотр..."
8,"Иски, связанные с возмещением ущерба → О взыск...",Александрович Валентин Петрович,01.08.2019,"ООО ""СК АНГАРА"", ООО «Группа Ренессанс Страхов...",2-2246/2019 ~ М-833/2019,Лебедева Александра Станиславовна,Кировский районный суд города Санкт-Петербурга,Рассмотрение дела начато с начала
9,"Иски, связанные с возмещением ущерба → О взыск...",Потапова Алиса Андреевна,01.08.2019,"СПАО ""РЕСО-Гарантия""",2-535/2018 (2-6702/2017;) ~ М-6879/2017,Семенова Ольга Алексеевна,Невский районный суд города Санкт-Петербурга,Отложено


In [31]:
df_9 = pd.read_excel('Spb_info_9.xlsx')
df_9

,place,date,id,category,clainmant,defendant,name_judge,result
0,Санкт-Петербургский городской суд,02.09.2019,33-17866/2019,"Иски, связанные с возмещением ущерба → О взыск...",Литвиненко Татьяна Александровна,СПАО ИнгосстрахСуд (мировой судья) первой инст...,Мелешко Наталья Владимировна,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
1,Красносельский районный суд города Санкт-Петер...,02.09.2019,2-1466/2019 (2-7279/2018;) ~ М-5904/2018,"Иски, связанные с возмещением ущерба → О взыск...",Артамонов Александр Валентинович,АО Группа Ренессанс страхование,Полинова Т.А.,Вынесено решение по делу
2,Московский районный суд города Санкт-Петербурга,02.09.2019,2-5020/2019 ~ М-2743/2019,"Иски, связанные с возмещением ущерба → О взыск...",Петровский Валентин Петрович,"АО ""МАКС""",Виноградова Ольга Евгеньевна,Заседание отложено
3,Приморский районный суд города Санкт-Петербурга,02.09.2019,2-5290/2019 ~ М-1262/2019,"Иски, связанные с возмещением ущерба → О взыск...",Рассказов Валерий Вячеславович,СПАО «РЕСО-Гарантия»,Тумасян Каринэ Левоновна,Назначено судебное заседание
4,Приморский районный суд города Санкт-Петербурга,02.09.2019,2-646/2020 (2-8050/2019;) ~ М-4430/2019,"Иски, связанные с возмещением ущерба → О взыск...",Ершов Роман Игоревич,СПАО «РЕСО-Гарантия»,Тумасян Каринэ Левоновна,Производство по делу приостановлено
5,Приморский районный суд города Санкт-Петербурга,02.09.2019,2-8713/2019 ~ М-5270/2019,"Иски, связанные с возмещением ущерба → О взыск...",Иванов Кирилл Игоревич,СПАО «РЕСО-Гарантия»,Тумасян Каринэ Левоновна,Заседание отложено
6,Приморский районный суд города Санкт-Петербурга,02.09.2019,2-1558/2019 (2-9688/2018;) ~ М-7555/2018,"Иски, связанные с возмещением ущерба → О взыск...",Шурупов Виталий Александрович,СПАО «РЕСО-Гарантия»,Тумасян Каринэ Левоновна,"Иск (заявление, жалоба) оставлены без рассмотр..."
7,Приморский районный суд города Санкт-Петербурга,02.09.2019,2-8894/2019 ~ М-5403/2019,"Иски, связанные с возмещением ущерба → О взыск...",Желиховский Роман Сергеевич,СПАО «РЕСО-Гарантия»,Тумасян Каринэ Левоновна,"Иск (заявление, жалоба) оставлены без рассмотр..."
8,Приморский районный суд города Санкт-Петербурга,02.09.2019,2-639/2020 (2-8020/2019;) ~ М-4529/2019,"Иски, связанные с возмещением ущерба → О взыск...",Петров Никита Витальевич,"Надирашвили Станислав Сергеевич, СПАО «РЕСО-Га...",Тумасян Каринэ Левоновна,Производство по делу приостановлено
9,Приморский районный суд города Санкт-Петербурга,02.09.2019,2-656/2020 (2-8080/2019;) ~ М-4546/2019,"Иски, связанные с возмещением ущерба → О взыск...",Черемных Николай Викторович,"САО «ВСК», СПАО «РЕСО-Гарантия»",Тумасян Каринэ Левоновна,Заседание отложено


In [32]:
df_10_12 = pd.read_excel('Spb_info_10-12.xlsx')
df_10_12

,category,clainmant,date,defendant,id,name_judge,place,result
0,"Иски, связанные с возмещением ущерба → О взыск...",Лебедев Виктор Иванович,01.10.2019,"АО ""АльфаСтрахование"", ПАО СК ""Росгосстрах""",2-3415/2019 ~ М-2172/2019,Лебедева Александра Станиславовна,Кировский районный суд города Санкт-Петербурга,Вынесено решение по делу
1,"Иски, связанные с возмещением ущерба → О взыск...",Лебедев Виктор Иванович,01.10.2019,"АО ""АльфаСтрахование""",2-1032/2019 (2-6087/2018;) ~ М-5203/2018,Лебедева Александра Станиславовна,Кировский районный суд города Санкт-Петербурга,Вынесено решение по делу
2,"Иски, связанные с возмещением ущерба → О взыск...",Туранский Игорь Викторович,01.10.2019,"АО ""МАКС""",2-2945/2019 ~ М-1545/2019,Рябинин Алексей Николаевич,Красногвардейский районный суд города Санкт-Пе...,Рассмотрение дела начато с начала
3,"Иски, связанные с возмещением ущерба → О взыск...",Оськин Роман Викторович,01.10.2019,"ООО ""Росгосстрах""",2-3458/2019 ~ М-2168/2019,Новикова Надежда Владимировна,Красносельский районный суд города Санкт-Петер...,Заседание отложено
4,"Иски, связанные с возмещением ущерба → О взыск...",Аубикиров Александр Зарифович,01.10.2019,"ПАО СК ""Росгосстрах""",2-596/2019 (2-6159/2018;) ~ М-4595/2018,Корнильева Светлана Алексеевна,Красносельский районный суд города Санкт-Петер...,Ходатайство/заявление УДОВЛЕТВОРЕНО
5,"Иски, связанные с возмещением ущерба → О взыск...",Игнатьев Илья Сергеевич,01.10.2019,"АО ""Группа Ренессанс Страхование"" филиал по СП...",2-33/2020 (2-443/2019;) ~ М-415/2019,Белолипецкий Андрей Александрович,Кронштадтский районный суд города Санкт-Петерб...,Заседание отложено
6,"Иски, связанные с возмещением ущерба → О взыск...",Ямс Надежда Константиновна,01.10.2019,"СПАО ""Ингосстрах""",2-2729/2019 ~ М-1787/2019,~Никитина Наталья Андреевна,Петроградский районный суд города Санкт-Петерб...,Объявлен перерыв
7,"Иски, связанные с возмещением ущерба → О взыск...",Кудряшова Екатерина Владимировна,01.10.2019,"СПАО ""Ингосстрах""",11-188/2019,Байбакова Татьяна Сергеевна,Петроградский районный суд города Санкт-Петерб...,Дело окончено
8,"Иски, связанные с возмещением ущерба → О взыск...",Колесник Максим Викторович,01.10.2019,ПАО СК «Росгосстрах»,2-2005/2019 (2-10335/2018;) ~ М-8296/2018,Тумасян Каринэ Левоновна,Приморский районный суд города Санкт-Петербурга,Заседание отложено
9,"Иски, связанные с возмещением ущерба → О взыск...",Силина Людмила Дмитриевна,01.10.2019,ПАО СК «Росгосстрах»,2-8506/2019 ~ М-4613/2019,Тумасян Каринэ Левоновна,Приморский районный суд города Санкт-Петербурга,"Иск (заявление, жалоба) оставлены без рассмотр..."


In [41]:
df = pd.concat( [df_1, df_2, df_7, df_8, df_9, df_10_12], ignore_index=True)
print(df)

                                               category  \
0     Иски, связанные с возмещением ущерба → О взыск...   
1     Иски, связанные с возмещением ущерба → О взыск...   
2     Иски, связанные с возмещением ущерба → О взыск...   
3     Иски, связанные с возмещением ущерба → О взыск...   
4     Иски, связанные с возмещением ущерба → О взыск...   
5     Иски, связанные с возмещением ущерба → О взыск...   
6     Иски, связанные с возмещением ущерба → О взыск...   
7     Иски, связанные с возмещением ущерба → О взыск...   
8     Иски, связанные с возмещением ущерба → О взыск...   
9     Иски, связанные с возмещением ущерба → О взыск...   
10    Иски, связанные с возмещением ущерба → О взыск...   
11    Иски, связанные с возмещением ущерба → О взыск...   
12    Иски, связанные с возмещением ущерба → О взыск...   
13    Иски, связанные с возмещением ущерба → О взыск...   
14    Иски, связанные с возмещением ущерба → О взыск...   
15    Иски, связанные с возмещением ущерба → О взыск... 

In [42]:
df

,category,clainmant,date,defendant,id,name_judge,place,result
0,"Иски, связанные с возмещением ущерба → О взыск...",Тамразова Джульетта Аршаковна,09.01.2019,"ОАО ""АльфаСтрахование""",2-50/2019 (2-2363/2018;) ~ М-1279/2018,Прозорова Елена Вадимовна,Василеостровский районный суд города Санкт-Пет...,Производство по делу приостановлено
1,"Иски, связанные с возмещением ущерба → О взыск...",Лебедева Ирина Павловна,09.01.2019,"АО ""Группа Ренессанс страхование""",2-1634/2019 (2-7780/2018;) ~ М-6289/2018,Метелкина Антонина Юрьевна,Московский районный суд города Санкт-Петербурга,Назначено судебное заседание
2,"Иски, связанные с возмещением ущерба → О взыск...",Алескерова Роза Исмаиловна,09.01.2019,"ПАО СК ""Росгосстрах""",11-2/2019 (11-115/2018;),Пазюченко Ирина Жоржовна,Октябрьский районный суд города Санкт-Петербурга,Дело окончено
3,"Иски, связанные с возмещением ущерба → О взыск...",Горохова Солмаз Эйвазовна,09.01.2019,"ПАО СК «Росгосстрах», Шварц Алексей Александрович",2-16/2019 (2-3432/2018; 2-14052/2017;) ~ М-118...,Тумасян Каринэ Левоновна,Приморский районный суд города Санкт-Петербурга,Заседание отложено
4,"Иски, связанные с возмещением ущерба → О взыск...",Мурская Ксения Вадимовна,09.01.2019,"ПАО СК ""Росгосстрах""",2-21/2019 (2-3710/2018; 2-14348/2017;) ~ М-115...,Мозерова Татьяна Михайловна,Приморский районный суд города Санкт-Петербурга,Вынесено решение по делу
5,"Иски, связанные с возмещением ущерба → О взыск...",Богданов Ю.С.,10.01.2019,"ООО ""Группа Ренессанс Страхование""Суд (мировой...",33-486/2019 (33-25834/2018;),Савельева Татьяна Юрьевна,Санкт-Петербургский городской суд,другие апелляционные определения с удовлетворе...
6,"Иски, связанные с возмещением ущерба → О взыск...",Велиева Э.М.,10.01.2019,"СПАО ""РЕСО-Гарантия""Суд (мировой судья) первой...",33-732/2019 (33-26404/2018;),Савельева Татьяна Юрьевна,Санкт-Петербургский городской суд,ОПРЕДЕЛЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
7,"Иски, связанные с возмещением ущерба → О взыск...",Жустров Юрий Аркадьевич,10.01.2019,"ПАО СК ""Росгосстрах""Суд (мировой судья) первой...",33-479/2019 (33-25818/2018;),Птохова Злата Юрьевна,Санкт-Петербургский городской суд,решение (не осн. требов.) изменено (без направ...
8,"Иски, связанные с возмещением ущерба → О взыск...",Котова М.А.,10.01.2019,"АО ""Либерти страхование""Суд (мировой судья) пе...",33-333/2019 (33-24990/2018;),Птохова Злата Юрьевна,Санкт-Петербургский городской суд,РЕШЕНИЕ оставлено БЕЗ ИЗМЕНЕНИЯ
9,"Иски, связанные с возмещением ущерба → О взыск...",Степанян В.А.,10.01.2019,"ПАО СМК ""Росгосстрах""Суд (мировой судья) перво...",33-959/2019 (33-26776/2018;),Александрова Юлия Кирилловна,Санкт-Петербургский городской суд,решение (осн. требов.) отменено полностью с вы...


In [43]:
df.to_excel('Spb_info.xlsx')